__DSC 478__

__Jonathan Lynch__

# Assignment 2 - Part 1:

## K-Nearest-Neighbor (KNN) classification on Newsgroups Dataset:

__Import libraries and read in necessary files:__

In [1]:
## load libraries
import sys
from numpy import *
import numpy as np
import pandas as pd
import operator

%cd "/Users/jonathanlynch/Desktop/DSC 478"

# import train & test matrices:
train_matrix = pd.read_csv('trainMatrixModified.txt', sep = "\t", header = None)
test_matrix = pd.read_csv('testMatrixModified.txt', sep = "\t", header = None)


# import train and test class labels:
train_labels = pd.read_csv('trainClasses.txt', sep = "\t", header = None)
test_labels = pd.read_csv('testClasses.txt', sep = "\t", header = None)


# import terms:
terms = pd.read_csv('modifiedterms.txt', header = None)

/Users/jonathanlynch/Desktop/DSC 478


__Transpose train_matrix & test_matrix:__

In [2]:
train_matrix_T = train_matrix.transpose()
test_matrix_T = test_matrix.transpose()

__Convert to arrays:__

In [3]:
train_matrix_T = np.array(train_matrix_T)
test_matrix_T = np.array(test_matrix_T)

train_labels = np.asarray(train_labels[1])
test_labels = np.asarray(test_labels[1])

## 1-A: Create KNN classifier function (inputs: instance, training data, training labels, K, distance measure):

In [4]:
def knn_classify(x, D, L, K, measure):
    if measure == 0:                        # euclidean dist
        dists = sqrt(((D - x)**2).sum(axis=1))
    elif measure == 1:                      # cosine dist
        D_norm = array([linalg.norm(D[i]) for i in range(len(D))])
        x_norm = linalg.norm(x)
        cosines = np.dot(D,x)/(D_norm * x_norm)
        dists = 1 - cosines
    idx = np.argsort(dists)
    count={}          
    for i in range(K):
        vote = L[idx[i]]
        count[vote] = count.get(vote,0) + 1
    sortedClass = sorted(count.items(), key = operator.itemgetter(1), reverse = True)
    return sortedClass[0][0], idx[:K]

__Look at two examples of the class prediction & k nearest neighbor indices with k=5 (Euclidean distance & Cosine similarity):__

In [5]:
class_prediction, neigh_indices = knn_classify(test_matrix_T[0], train_matrix_T, train_labels, 5, 0) # Euclidean dist
print('Euclidean Distance:')
print('class prediction: ' + str(class_prediction))
print('k nearest neighbor indices: ' + str(neigh_indices))
print()
class_prediction2, neigh_indices2 = knn_classify(test_matrix_T[0], train_matrix_T, train_labels, 5, 1) # Cosine similarity
print('Cosine Similarity:')
print('class prediction: ' + str(class_prediction2))
print('k nearest neighbor indices: ' + str(neigh_indices2))

Euclidean Distance:
class prediction: 1
k nearest neighbor indices: [685 628 703 266 510]

Cosine Similarity:
class prediction: 1
k nearest neighbor indices: [685 628 667 152 427]


## 1-B: Create a function to compute the classification accuracy over all the test data:

In [6]:
def accuracy(T, x, D, L, K, measure):
    num_labels = len(T)
    errors = 0.0
    for i in range(num_labels):
        class_prediction, neigh_indices = knn_classify(x[i,:], D, L, K, measure)
        if (class_prediction != T[i]): 
            errors += 1.0
    error_rate = errors/float(num_labels)
    return (1-error_rate)

__Check the accuracy of the knn classifier over the entire test dataset for both distance measures (with k=5):__

In [7]:
result1 = accuracy(test_labels, test_matrix_T, train_matrix_T, train_labels, 5, 0)
print('The accuracy rate with Euclidean distance is: ' + str(result1))

result2 = accuracy(test_labels, test_matrix_T, train_matrix_T, train_labels, 5, 1)
print('The accuracy rate with Cosine similarity is: ' + str(result2))

The accuracy rate with Euclidean distance is: 0.815
The accuracy rate with Cosine similarity is: 0.97


## 1-C: Run accuracy function on a range of values for K and both distance measures in order to compare differences in accuracy rates:

In [8]:
accuracies = np.zeros((20,3), dtype=float)
for i in range(20):
    accuracy_Euc = accuracy(test_labels, test_matrix_T, train_matrix_T, train_labels, i+1, 0)
    accuracy_Cos = accuracy(test_labels, test_matrix_T, train_matrix_T, train_labels, i+1, 1)
    accuracies[i] = [i+1, accuracy_Euc, accuracy_Cos]
    
print('Differences in Accuracy:\n')
print(" K  Euclid  Cosine")
for row in accuracies:
    print("%2.0f   %.2f    %.2f" % (row[0], row[1], row[2]))

Differences in Accuracy:

 K  Euclid  Cosine
 1   0.78    0.98
 2   0.78    0.98
 3   0.81    0.97
 4   0.81    0.98
 5   0.81    0.97
 6   0.83    0.98
 7   0.77    0.98
 8   0.80    0.98
 9   0.75    0.97
10   0.85    0.98
11   0.80    0.98
12   0.84    0.97
13   0.78    0.98
14   0.81    0.98
15   0.79    0.98
16   0.80    0.98
17   0.76    0.97
18   0.79    0.97
19   0.74    0.97
20   0.77    0.97


## 1-D: Convert term weights to TFxIDF weights and rerun evaluation on range of values for K with both distance measures:

__converting train_matrix to TFxIDF weights:__ 

In [9]:
TD = train_matrix                        # renaming train_matrix
DF = pd.DataFrame([(TD!=0).sum(1)]).T    # doc counts for each term
NDocs = TD.shape[1]                      # num of docs
NMatrix=np.ones(np.shape(TD), dtype=float)*NDocs   # create matrix  with all entries = num docs
IDF = np.log2(np.divide(NMatrix, np.array(DF)))    # convert each entry to IDF value
TD_tfidf = TD * IDF                      # compute the TFxIDF values for each document-term entry
DT_tfidf = TD_tfidf.T                    # transposing to doc x term matrix
DT_array = np.array(DT_tfidf)            # converting to array

__converting test_matrix to TFxIDF weights:__

In [10]:
TD1 = test_matrix                        # renaming test_matrix
DF1 = pd.DataFrame([(TD1!=0).sum(1)]).T    # doc counts for each term
DF1.replace(0,1,inplace=True)              # replace all zeros with ones so don't get an error when trying to divide by a zero when computing IDF1
NDocs1 = TD1.shape[1]                      # num of docs
NMatrix1=np.ones(np.shape(TD1), dtype=float)*NDocs1   # create matrix  with all entries = num docs
IDF1 = np.log2(np.divide(NMatrix1, np.array(DF1)))    # convert each entry to IDF value
TD_tfidf1 = TD1 * IDF1                      # compute the TFxIDF values for each document-term entry
DT_tfidf1 = TD_tfidf1.T                    # transposing to doc x term matrix
DT_array1 = np.array(DT_tfidf1)            # converting to array

__Re-run accuracy function with test and training sets converted to TFxIDF weights on a range of values for K:__

In [11]:
TFIDF_accuracies = np.zeros((20,3), dtype=float)
for i in range(20):
    accuracy_Euc = accuracy(test_labels, DT_array1, DT_array, train_labels, i+1, 0)
    accuracy_Cos = accuracy(test_labels, DT_array1, DT_array, train_labels, i+1, 1)
    TFIDF_accuracies[i] = [i+1, accuracy_Euc, accuracy_Cos]
    
print('Differences in Accuracy with TFxIDF weights:\n')
print(" K  Euclid  Cosine")
for row in TFIDF_accuracies:
    print("%2.0f   %.2f    %.2f" % (row[0], row[1], row[2]))

Differences in Accuracy with TFxIDF weights:

 K  Euclid  Cosine
 1   0.71    0.95
 2   0.71    0.95
 3   0.73    0.96
 4   0.71    0.97
 5   0.80    0.97
 6   0.82    0.97
 7   0.71    0.98
 8   0.76    0.99
 9   0.66    0.98
10   0.75    0.99
11   0.69    0.99
12   0.72    0.99
13   0.65    0.99
14   0.66    0.99
15   0.61    0.99
16   0.67    0.99
17   0.61    0.99
18   0.67    0.99
19   0.61    0.99
20   0.67    0.99


Comparing the accuracy results of the training and test sets converted to TFxIDF weights with the accuracy results of the training and test sets using raw term frequencies, the Euclidean distance measure performed worse (less accuracy) across all values of K with the TFxIDF weighted training/test sets.  Using the Cosine similarity measure, the accuracy results were relatively good for both TFxIDF weighted training/test sets and training/test sets with raw term frequencies.  It looks like for lower values of K, using the raw term frequencies for the training/test sets performed slightly better, and for higher values of K, training/test sets converted to TFxIDF weights were slightly more accurate for the Cosine similarity measure.

## 1-E: Create a new classifier based on the Rocchio Method adapted for text categorization:

__Create Rocchio training function that returns prototype vectors for each class:__

In [12]:
train_labels = pd.read_csv('trainClasses.txt', sep = "\t", header = None)
DT_TFIDF = TD_tfidf.T

def rocchio_train(DT_TFIDF, train_labels):
    hockey_idx = train_labels[(train_labels[1] == 1)]
    windows_idx = train_labels[(train_labels[1] == 0)]  
    hindex = hockey_idx.index 
    windex = windows_idx.index
    windows_proto_DF = DT_TFIDF[DT_TFIDF.index.isin(windex)]
    windows_proto_vector = windows_proto_DF.sum()
    hockey_proto_DF = DT_TFIDF[DT_TFIDF.index.isin(hindex)]
    hockey_proto_vector = hockey_proto_DF.sum()
    return windows_proto_vector, hockey_proto_vector

__Create Rocchio classifier function that returns class prediction & similarity values to each category prototype:__

In [13]:
def rocchio_classifier(windows_proto_vector,hockey_proto_vector,ins):
    x = windows_proto_vector
    y = hockey_proto_vector
   
    # compute the cosine similarity between ins and both x and y
    x_norm = np.linalg.norm(x)
    y_norm = np.linalg.norm(y) 
    ins_norm = np.linalg.norm(ins)    
    
    # x (windows)
    xsim = np.dot(x,ins)/(x_norm * ins_norm)
    xdist = 1 - xsim
    
    # y (hockey)
    ysim = np.dot(y,ins)/(y_norm * ins_norm)
    ydist = 1 - ysim
    
    # return the predicted class
    if xdist < ydist:
        pred_class = 0
    elif ydist < xdist:
        pred_class = 1


    # return the similarity values of the instance to each of the category prototypes & class prediction
    return xsim, ysim, pred_class

__Create function to compute the classification accuracy over all the test data using Rocchio method:__

In [14]:
def rocchio_accuracy(T, ins, D, L):
    num_labels = len(T)
    errors = 0.0
    windows_proto_vector, hockey_proto_vector = rocchio_train(DT_TFIDF, train_labels)
    for i in range(num_labels):
        xsim, ysim, pred_class = rocchio_classifier(windows_proto_vector,hockey_proto_vector,test_matrix_T[i,:])
        if (pred_class != T[i]): 
            errors += 1.0
    error_rate = errors/float(num_labels)
    return (1-error_rate)

__Compute the Rocchio classification accuracy using the test instances and compare with previous results:__

In [15]:
result_rocchio = rocchio_accuracy(test_labels, test_matrix_T, DT_TFIDF, train_labels)
print('The accuracy rate with the Rocchio Method is: ' + str(result_rocchio))

The accuracy rate with the Rocchio Method is: 0.98


The accuracy rate with the Rocchio Method adapted for text categorization is 98%.  This is a very high accuracy rate, and compared to the results of the approaches tried earlier, seems to be on par with some of the very best results.  The highest accuracy measure recorded was 99% using a Cosine similarity measure in KNN with TFxIDF weighted training/test sets (where the value of K was 8 to 20).  Thus, it seems that the Rocchio method compares very well.